<a href="https://colab.research.google.com/github/Tarunika21/HiringManagement/blob/main/Hiring_Management.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [163]:
!pip install pdfplumber
!pip install pytesseract
!sudo apt install tesseract-ocr
!pip install pdf2image
!sudo apt-get update
!sudo apt-get install python-poppler
!pip install PyMuPDF
!pip install Aspose.Email-for-Python-via-NET
!pip install aspose-words
!pip install tqdm

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 109 kB in 3s (41.8 kB/s)

In [197]:
from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec
import pandas as pd
import joblib
import os
import warnings
import string
import numpy as np
from itertools import groupby, count
import re
import subprocess
import os.path
import logging
import pytesseract
import cv2
from pdf2image import convert_from_path
from PIL import Image
import aspose.words as aw
import fitz
Image.MAX_IMAGE_PIXELS = 1000000000
logger_watchtower = logging.getLogger(__name__)
warnings.filterwarnings(action = 'ignore')

In [198]:
# csv file with data and url from it was taken
df = pd.read_csv('/content/domain_knowledge.csv')
df.head(2)

,Unnamed: 0,URL,data
0,0,https://www.analyticsvidhya.com/blog/2022/06/c...,article published part data science blogathon ...
1,1,https://www.analyticsvidhya.com/blog/2022/06/2...,article published part data science blogathon ...


In [199]:
sent = [row.split() for row in df['data']]  # tokenize
phrases = Phrases(sent, min_count=30, progress_per=10000)
sentences = phrases[sent]

In [200]:
# word embeddings
w2v_model = Word2Vec(min_count=20,     # ignore than this count
                     window=3,         # max of this doesn't belong to same group
                     vector_size=300,  # embedding size
                     sample=6e-5,
                     alpha=0.03,
                     min_alpha=0.0007,
                     negative=20
                     )

In [201]:
# building vocabulary
w2v_model.build_vocab(sentences)

In [202]:
# words in the model
w2v_keys = w2v_model.wv.index_to_key

In [203]:
# training the model
w2v_model.train(
    sentences,
    total_examples = w2v_model.corpus_count,
    epochs = 30,
    report_delay = 1
    )

(648622, 2906040)

In [204]:
# save the model
joblib.dump(w2v_model, 'm1.joblib')

['m1.joblib']

In [205]:
# checking for similarity of related words
model1 = joblib.load('m1.joblib')
print(model1.wv.similarity('neural_network', 'machine_learning'))

0.66658497


In [206]:
# identify the boxes and return the skills set in it

def _skills_in_box(image_gray,threshold=60):
  img = image_gray.copy()
  thresh_inv = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)[1]

  # Blur the image
  blur = cv2.GaussianBlur(thresh_inv,(1,1),0)
  thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]

  # find contours
  contours = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
  mask = np.ones(img.shape[:2], dtype="uint8") * 255
  available = 0

  for c in contours:
    # get the bounding rect
    x, y, w, h = cv2.boundingRect(c)
    if w*h>1000:
        cv2.rectangle(mask, (x+5, y+5), (x+w-5, y+h-5), (0, 0, 255), -1)
        available = 1

  res = ''
  if available == 1:
    res_final = cv2.bitwise_and(img, img, mask=cv2.bitwise_not(mask))
    res_final[res_final<=threshold]=0
    kernel = np.array([[0, -1, 0], [-1, 5,-1], [0, -1, 0]])
    res_fin = cv2.filter2D(src=res_final, ddepth=-1, kernel=kernel)
    vt = pytesseract.image_to_data(255-res_final,output_type='data.frame')
    vt = vt[vt.conf != -1]
    res = ''

    for i in vt[vt['conf']>=43]['text']:
      res = res + str(i) + ' '
  print(res)
  return res

In [207]:
# return text in an image

def _image_to_string(img):
  img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  res = ''
  string1 = pytesseract.image_to_data(img,output_type='data.frame')
  string1 = string1[string1['conf'] != -1]

  for i in string1[string1['conf']>=43]['text']:
    res = res + str(i) + ' '
  string3 = _skills_in_box(img)
  return res+string3

In [208]:
# converts the pdf to image
# return the text in it as string

def _pdf_to_png(pdf_path):
    string = ''
    images = convert_from_path(pdf_path)

    for j in tqdm(range(len(images))):
        # Save pages as images in the pdf
        image = np.array(images[j])
        string += _image_to_string(image)
        string += '\n'
    return string


In [209]:
# to check if the pdf is image or not.
# if .doc converts it to .pdf

def ocr(paths):
    text = ""
    res = ""

    try:
        doc = fitz.open(paths)
        for page in doc:
            text += page.get_text()
        if len(text) <=10 :
            res = _pdf_to_png(paths)
        else:
            res = text
    except:
        doc = aw.Document(paths)
        doc.save("Document.pdf")
        doc = fitz.open("Document.pdf")
        for page in doc:
            text += page.get_text()
        if len(text) <=10 :
            res = _pdf_to_png("Document.pdf")
        else:
            res = text
        os.remove("Document.pdf")
    return res

In [210]:
# to add an axis
def to_la(L):
  k=list(L)
  l=np.array(k)

  return l.reshape(-1, 1)

# to find cosine similarity
def cos(A, B):
  dot_prod=np.matmul(A,B.T)
  norm_a=np.reciprocal(np.sum(np.abs(A)**2,axis=-1)**(1./2))
  norm_b=np.reciprocal(np.sum(np.abs(B)**2,axis=-1)**(1./2))
  norm_a=to_la(norm_a)
  norm_b=to_la(norm_b)
  k=np.matmul(norm_a,norm_b.T)

  return list(np.multiply(dot_prod,k))

In [211]:
# to screen the job description skills
# with skills in the resume

def check(path,skills,l2,w2v_model1,phrases,pattern):
  text = ocr(path)
  text = re.sub(r'[^\x00-\x7f]',r' ',text)
  text = text.lower()
  text = re.sub("\\\|,|/|:|\)|\("," ",text)
  t2 = text.split()
  l_2=l2.copy()
  match=list(set(re.findall(pattern,text)))
  sentences=phrases[t2]
  resume_skills_dict={}
  res_jdskill_intersect=list(set(sentences).intersection(set(l_2)))

  if(len(match)!=0):
    for k in match:
      k=k.replace(' ','_')
      resume_skills_dict[k]=1
      try:
        l_2.remove(k)
      except:
        continue

  l6=list(set(l_2).intersection(skills[0]))
  l6_minus_skills=list(set(l_2).difference(skills[0]))

  for i in l6_minus_skills:
    resume_skills_dict[i]=0

  if(len(l6)==0):
    return resume_skills_dict

  l4=list(set(sentences).intersection(skills[0]))
  arr1=np.array([w2v_model1[i] for i in l6])
  arr2=np.array([w2v_model1[i] for i in l4])
  similarity_values=cos(arr1,arr2)
  count=0

  # setting threshold of 0.38
  for i in similarity_values:
    k=list(filter(lambda x: x<0.38, list(i)))
    if(len(k)==len(i)):
      resume_skills_dict[l6[count]]=0
    else:
      resume_skills=[s for s in range(len(i)) if(i[s])>0.38]
      resume_skills_dict[l6[count]]=1
    count+=1

  return resume_skills_dict

In [212]:
def preprocess(string):
  string = string.replace(",",' ')
  string= string.replace("'",' ')
  string = list(set(list(string.split())))
  return string

In [213]:
# driver function

w2v_model1 = joblib.load('/content/m1.joblib')  # loading the model
skills = w2v_keys    # loading the vocabulary
jd_skills = 'java, kaggle, ml, mysql, oracle, python, pytorch, r, scikit learn, snowflake, sql, django'  # jd skills
resume_path = '/content/resume3.pdf'  # to test

# data preprocessing
mapper = {}
underscore=[]
jd_skills=" ".join(jd_skills.strip().split())
jd_skills=jd_skills.replace(', ',',')
pattern=jd_skills.replace(',','|').lower()

for i in jd_skills.split(','):
  if '_' in i:
    underscore.append(i)
    mapper[i.lower().replace('_',' ')] = i

jd_skills=jd_skills.replace(' ','_')
jd_skills=jd_skills.replace(',',', ')

for i in jd_skills.split(', '):
  if i not in underscore:
    if '_' in i:
      mapper[i.lower().replace('_',' ')] = i.replace('_',' ')
    elif '-' in i:
      mapper[i.lower().replace('-',' ')] = i
    else:
      mapper[i.lower()] = i

jd_skills=jd_skills.replace('-','_')
lines = [preprocess(jd_skills.lower().rstrip())]
final_jd_skills=list(set(lines[0]).intersection(skills[0]))

res = check(resume_path,skills,lines[0],w2v_model1,phrases,pattern)

print('skills_matched :',res)

skills_matched : {'r': 1, 'java': 1, 'sql': 1, 'python': 1, 'django': 1, 'ml': 1, 'scikit_learn': 0, 'kaggle': 0, 'snowflake': 0, 'oracle': 0, 'pytorch': 0, 'mysql': 0}
